In [2]:
import pandas as pd
import numpy as np
import ast

In [3]:
df = pd.read_csv("hasil_preprocessingdua.csv", usecols=["tweet_stemming","sentimen"])
df.columns = ["sentimen", "tweet"]
df.head(10)

,sentimen,tweet
0,netral,"['sehat', 'vaksin', 'kini', 'viralkan', 'cerit..."
1,negatif,"['sekarang', 'i', 'faham', 'kenapa', 'org', 't..."
2,negatif,"['tengok', 'doktor', 'pun', 'ada', 'yang', 'ce..."
3,netral,"['side', 'effects', 'vaksin', 'ini', 'kalau', ..."
4,positif,"['kisah', 'kami', 'positif', 'covid', 'di', 'b..."
5,negatif,"['bila', 'orang', 'pakar', 'ubat', 'famili', '..."
6,negatif,"['lately', 'ini', 'banyak', 'dengar', 'orang',..."
7,negatif,"['frontliner', 'dapat', 'pfizer', 'kami', 'dap..."
8,negatif,"['berat', 'lah', 'sek', 'ini', 'menate', 'mano..."
9,positif,"['antara', 'negara', 'keluar', 'vaksin', 'dan'..."


In [4]:
#convert string to list
def convert(tweets):
    tweets = ast.literal_eval(tweets)
    return [tweet for tweet in tweets]

df["list_tweet"] = df["tweet"].apply(convert)
print(df["list_tweet"])

print("\ntype : ", type(df["list_tweet"]))

0      [sehat, vaksin, kini, viralkan, cerita, benar,...
1      [sekarang, i, faham, kenapa, org, tua, tak, da...
2      [tengok, doktor, pun, ada, yang, cerita, pasal...
3      [side, effects, vaksin, ini, kalau, boleh, mut...
4      [kisah, kami, positif, covid, di, bumi, englan...
                             ...                        
297    [tau, kenapa, pm, bodo, tu, cakap, dia, dah, 2...
298    [lepas, ini, tugas, yang, suntik, vaksin, akan...
299                                [aku, reject, vaksin]
300    [hanya, pharmaniaga, yang, bekal, vaksin, sinn...
301    [tiada, monopoli, rana, hanya, ada, satu, syar...
Name: list_tweet, Length: 302, dtype: object

type :  <class 'pandas.core.series.Series'>


In [5]:
#tf 
def calc_TF(document):
    # Counts the number of times the word appears in review
    TF_dict = {}
    for term in document:
        if term in TF_dict:
            TF_dict[term] += 1
        else:
            TF_dict[term] = 1
    # Computes tf for each word
    for term in TF_dict:
        TF_dict[term] = TF_dict[term] / len(document)
    return TF_dict

df["TF_dict"] = df['list_tweet'].apply(calc_TF)

df["TF_dict"].head(10)

0    {'sehat': 0.08333333333333333, 'vaksin': 0.083...
1    {'sekarang': 0.045454545454545456, 'i': 0.0909...
2    {'tengok': 0.09090909090909091, 'doktor': 0.09...
3    {'side': 0.125, 'effects': 0.125, 'vaksin': 0....
4    {'kisah': 0.05555555555555555, 'kami': 0.05555...
5    {'bila': 0.05, 'orang': 0.05, 'pakar': 0.05, '...
6    {'lately': 0.05555555555555555, 'ini': 0.05555...
7    {'frontliner': 0.05263157894736842, 'dapat': 0...
8    {'berat': 0.045454545454545456, 'lah': 0.04545...
9    {'antara': 0.07692307692307693, 'negara': 0.30...
Name: TF_dict, dtype: object

In [6]:
# Check TF result
#index = 1

#print('%20s' % "term", "\t", "TF\n")
#for key in df["TF_dict"][index]:
 #   print('%20s' % key, "\t", df["TF_dict"][index][key])

In [7]:
#idf
def calc_DF(tfDict):
    count_DF = {}
    # Run through each document's tf dictionary and increment countDict's (term, doc) pair
    for document in tfDict:
        for term in document:
            if term in count_DF:
                count_DF[term] += 1
            else:
                count_DF[term] = 1
    return count_DF

DF = calc_DF(df["TF_dict"])

In [8]:
n_document = len(df)

def calc_IDF(__n_document, __DF):
    IDF_Dict = {}
    for term in __DF:
        IDF_Dict[term] = np.log(__n_document / (__DF[term] + 1))
    return IDF_Dict
  
#Stores the idf dictionary
IDF = calc_IDF(n_document, DF)

In [39]:
#calc TF-IDF
def calc_TF_IDF(TF):
    TF_IDF_Dict = {}
    #For each word in the review, we multiply its tf and its idf.
    for key in TF:
        TF_IDF_Dict[key] = TF[key] * IDF[key]
    return TF_IDF_Dict

#Stores the TF-IDF Series
df["TF-IDF_dict"] = df["TF_dict"].apply(calc_TF_IDF)
print(df["TF-IDF_dict"].head(10))

0    {'sehat': 0.3137097390266297, 'vaksin': 0.0194...
1    {'sekarang': 0.18640859567912588, 'i': 0.33008...
2    {'tengok': 0.37281719135825175, 'doktor': 0.34...
3    {'side': 0.48983344351835184, 'effects': 0.540...
4    {'kisah': 0.2787377687119402, 'kami': 0.227832...
5    {'bila': 0.1882258434159778, 'orang': 0.115461...
6    {'lately': 0.2787377687119402, 'ini': 0.071089...
7    {'frontliner': 0.2427270909845663, 'dapat': 0....
8    {'berat': 0.22805817440067838, 'lah': 0.150569...
9    {'antara': 0.3547549791312892, 'negara': 1.080...
Name: TF-IDF_dict, dtype: object


In [27]:
# Cek hasil TF-IDF
#index = 90

#print('%20s' % "term", "\t", '%10s' % "TF", "\t", '%20s' % "TF-IDF\n")
#for key in df["TF-IDF_dict"][index]:
 #   print('%20s' % key, "\t", df["TF_dict"][index][key] ,"\t" , df["TF-IDF_dict"][index][key])

In [28]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [29]:
x = df['TF-IDF_dict']
y = df['sentimen']

In [30]:
#splitting data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)


print("total Training Data :", y_train.shape)
print("total Testing Data :", y_test.shape)

total Training Data : (241,)
total Testing Data : (61,)


In [31]:
y_train.value_counts(ascending=True)

netral     76
negatif    80
positif    85
Name: sentimen, dtype: int64

In [32]:
y_test.value_counts(ascending=True)

positif    15
negatif    21
netral     25
Name: sentimen, dtype: int64

In [37]:
nb = MultinomialNB().fit(x_train,y_train)

TypeError: float() argument must be a string or a number, not 'dict'